<a href="https://colab.research.google.com/github/PPareun/DeepLearningTextBook/blob/main/Keyword_Extractor/Keyword_From_SUM/TextBert_SUM_Implement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 from google.colab import drive
 drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers[sentencespiece]

In [3]:
import transformers

In [4]:
import torch

In [5]:
from sklearn.model_selection import train_test_split
from torch.optim import Adam
from torch.nn import functional as F
from transformers import BertTokenizer, BertModel

In [6]:
import pandas as pd

In [7]:
Train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')

In [9]:
Test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv')
Val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/validation.csv')

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower=True)

In [9]:
class DataFetch():
  def __init__(self, index):
    if index == 'Train':
      self.data = Train
    elif index == 'Val':
      self.data = Val
    elif index == 'Test':
      self.data = Test
  def extractor(self,index):
    article_texts = self.data['article'][index].strip().split('.')
    article_sentences = [(sent + '.') for sent in article_texts][:-1]
    sum_texts = self.data['highlights'][index].strip().split('\n')
    sum_sentences = [sent for sent in sum_texts]
    
    return article_sentences, sum_sentences

In [10]:
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
y = []

In [14]:
for i in range(len(Train)):
  art, sum = DataFetch(index = 'Train').extractor(i)
  count = 0
  y_data = [0] * len(art)
  for sum_ in sum:
    MAX = 0
    BEST = -1
    for index, art_ in enumerate(art):
      MAAX = MAX
      try:
        MAX = max(MAX,rouge.get_scores(art_,sum_, avg = True)["rouge-2"]["f"])
        if(MAX!=MAAX):
          BEST = index
      except:
        count += 1
    if(BEST>-1):
      y_data[BEST] = 1
  y.append(y_data)

In [12]:
class Embedding():
  def __init__(self, sentences):
    self.sentences = sentences
  def embedding(self):
    #Word Embedding
    word_embedding = [tokenizer.encode(sentence) for sentence in self.sentences]
    pad_size = 512
    word_emb = []
    count = 0
    #Masked CLS
    mask_cls = [0] * pad_size
    #Word Embedding to 1d
    for words in word_embedding:
      #length truncate
      if(count + len(words)>pad_size):
        break
      else:
        mask_cls[count] = 1
        count += len(words)
        word_emb.extend(words)

    #Segment Embedding
    segment_emb = []
    for index in range(len(word_embedding)):
      segment = [index%2] * len(word_embedding[index])
      segment_emb.extend(segment)

    #Padding
    padded_word_emb = word_emb + [tokenizer.pad_token_id] * (pad_size - count)
    padded_segment_emb = segment_emb[:count] + [tokenizer.pad_token_id] * (pad_size - count)
    attention_mask = [1]*count + [0]*(pad_size-count)
    clss = [i for i, d in enumerate(word_emb) if d == tokenizer.vocab["[CLS]"]]
    return torch.tensor(padded_word_emb), torch.tensor(padded_segment_emb), torch.tensor(attention_mask), torch.tensor(clss)

In [ ]:
word_emb, seg_emb, attention_mask, clss = Embedding(DataFetch('Train').extractor(3000)[0]).embedding()
print(seg_emb)

In [13]:
word_emb, seg_emb, attention_mask, clss = [], [], [], []

In [ ]:
"""for i in range(len(Train)):
  w,s,a,c = Embedding(DataFetch('Train').extractor(i)[0]).embedding()
  word_emb.append(w)
  seg_emb.append(s)
  attention_mask.append(a)
  clss.append(c)

In [6]:
import pickle
with open("/content/drive/MyDrive/word_emb.pkl","rb") as fw:
    word_emb = pickle.load(fw)
with open("/content/drive/MyDrive/seg_emb.pkl","rb") as fw:
    seg_emb = pickle.load(fw)
with open("/content/drive/MyDrive/attention_mask.pkl","rb") as fw:
    attention_mask = pickle.load(fw)
with open("/content/drive/MyDrive/clss.pkl","rb") as fw:
    clss = pickle.load(fw)
with open("/content/drive/MyDrive/y.pkl","rb") as fw:
    y = pickle.load(fw)

In [7]:
clss_ten = []
y_ten = []

In [8]:
for i in range(len(clss)):
  clss_ten.append(torch.cat([clss[i], torch.tensor([-1]*(512-clss[i].size(dim = 0)))], dim = 0))
  y_ten.append(torch.tensor(y[i][:512] + [0] * (512-len(y[i]))))

In [9]:
word_emb_tensor = torch.stack(word_emb, 0)
seg_emb_tensor = torch.stack(seg_emb, 0)
attention_mask_tensor = torch.stack(attention_mask, 0)
clss_tensor = torch.stack(clss_ten, 0)
y_tensor = torch.stack(y_ten, 0)

In [10]:
from torch.utils.data import TensorDataset, random_split
dataset = TensorDataset(word_emb_tensor, seg_emb_tensor, attention_mask_tensor, clss_tensor, y_tensor)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [11]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
batch_size = 16
train_dataloader = DataLoader(
            train_dataset, 
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size
        )
validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size
        )


In [12]:
import torch.nn as nn
from transformers import BertModel

In [13]:
import math
class PositionalEncoding(nn.Module):

    def __init__(self, dropout: float = 0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(512).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, 512, 2) * (-math.log(10000.0) / 512))
        pe = torch.zeros(512, 1, 512)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [20]:
class BERTSUM(nn.Module):
  def __init__(self):
    super(BERTSUM, self).__init__()
    self.bert = BertModel.from_pretrained("bert-base-uncased")
    self.pos_emb = PositionalEncoding()
    self.encoder = nn.TransformerEncoderLayer(d_model=512, nhead=8)
    self.linear = nn.Linear(512, 1, bias = True)
    self.sigmoid = nn.Sigmoid()
  def forward(self, word_emb, seg_emb, attn_mask, clss):
    output, _ = self.bert(input_ids = word_emb, attention_mask = attn_mask, token_type_ids = seg_emb)
    output = output[-1]
    output = output[torch.arange(output.size(0)).unsqueeze(1), clss]
    mask_cls = clss + [0] * (512-len(clss))
    output = output * mask_cls[:, :, None].float()
    output = output + self.pos_emb(output)
    output = self.encoder(output)
    output = self.encoder(output)
    output = self.linear(output)
    output = self.sigmoid()

    return output, mask_cls

In [19]:
A = torch.tensor([[1,2,3,4],[2,3,4,5],[3,4,5,6],[3,4,5,6],[3,4,5,6],[3,4,5,6]])
B = torch.tensor([[1,2,0],[2,3,0],[1,2,0],[1,2,0],[1,2,0],[1,2,0]])
C = torch.tensor([[1,1,0],[1,1,0],[1,1,0],[1,0,1],[0,0,1],[0,0,1]])
A[torch.arange(A.size(0)).unsqueeze(1), B] * C

tensor([[2, 3, 0],
        [4, 5, 0],
        [4, 5, 0],
        [4, 0, 3],
        [0, 0, 3],
        [0, 0, 3]])

In [ ]:
model = BERTSUM()
print(model)

In [16]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(),lr = 2e-5, eps = 1e-6)

In [17]:
from transformers import get_linear_schedule_with_warmup
epochs = 4
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
import random
import numpy as np
import time
seed_val = 777
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
training_stats = []
total_t0 = time.time()
model = BERTSUM()
lossF = nn.BCELoss(reduction='none')
for epoch_i in range(0, epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    t0 = time.time()
    total_train_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
        token = batch[0]
        segment = batch[1]
        attention = batch[2]
        clss = batch[3]
        label = batch[4]
        model.zero_grad()        
        pred, mask_cls = model(word_emb = token, seg_emb = segment, attn_mask = attention, clss = clss)
        loss = lossF(pred * mask_cls, label * mask_cls)
        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
    avg_train_loss = total_train_loss / len(train_dataloader)        
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
    print("")
    print("Running Validation...")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



======== Epoch 1 / 4 ========
Training...
